In [ ]:
import os
import cv2
import torch
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO

def detectx(frame, model):
    # Function to perform object detection using the YOLO model
    # Returns labels and coordinates of detected objects
    results = model(frame)
    labels, coordinates = results.xyxyn[0][:, -1], results.xyxyn[0][:, :-1]
    return labels, coordinates

def calculate_pointer_angle(img, coords):
    # Function to calculate pointer angles based on keypoints
    model1 = YOLO(model_path)
    results = model1(img)
    theta1 = None
    theta2 = None
    
    for result in results:
        keypoints_set = result.keypoints.xy.cpu().numpy()

        if keypoints_set.size == 0:
            print('No Key Point Detected')
            theta1 = 0
            theta2 = 45

        if keypoints_set.shape[1] < 4:
            print('Not enough keypoints detected')

        A, B, C, D = keypoints_set[0][0], keypoints_set[0][1], keypoints_set[0][3], keypoints_set[0][2]

        theta1 = compute_angle(A, B, C)
        theta2 = 360 - compute_angle(B, D, C)

        print(f"Angle between keypoints A, B, and C is: {theta1:.2f} degrees")
        print(f"Angle between keypoints B, C, and D is: {theta2:.2f} degrees")

    return theta1, theta2

def compute_angle(A, B, C):
    # Function to compute the angle between three points using the law of cosines
    a = [A[0] - B[0], A[1] - B[1]]
    b = [C[0] - B[0], C[1] - B[1]]
    c = [A[0] - C[0], A[1] - C[1]]

    ma = math.sqrt(a[0]**2 + a[1]**2)
    mb = math.sqrt(b[0]**2 + b[1]**2)
    mc = math.sqrt(c[0]**2 + c[1]**2)

    cosB = (ma**2 + mc**2 - mb**2) / (2 * ma * mc)
    angle = math.acos(cosB)
    angle_deg = math.degrees(angle)

    return angle_deg

def main(img_folder=None):
    print(f"[INFO] Loading YOLOv5 model... ")
    model_path = '/home/agrotech/deep-gauge-master/Reading_Circular_Analogue_Gauges_using_Digital_Image_Processing/yolov5-master/yolov5/demo_20feb_v1.pt'
    model = torch.hub.load('ultralytics/yolov5:v5.0', 'custom', source='local', path=model_path, force_reload=True)
    classes = model.names

    # Replace 'path/to/your/image/folder' with the actual image folder path
    image_folder = img_folder or 'path/to/your/image/folder'

    angle_readings = []

    for image_file in os.listdir(image_folder):
        if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(image_folder, image_file)

            # Load the image using OpenCV
            img = cv2.imread(image_path)

            # Perform object detection
            labels, coordinates = detectx(img, model)

            # Calculate pointer angles based on keypoints
            theta1, theta2 = calculate_pointer_angle(img, coordinates)

            # Append angle readings to the list
            angle_readings.append(theta1)

    # Store angle readings in Excel file
    df = pd.DataFrame({'Angle_Readings': angle_readings})
    df.to_excel('angle_readings.xlsx', index=False)

    # Plot a graph
    plt.plot(angle_readings)
    plt.title('Angle Readings vs Number of Images')
    plt.xlabel('Number of Images')
    plt.ylabel('Angle Readings')
    plt.show()

if __name__ == "__main__":
    main()
